In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os

In [2]:
from Hybrid import scrape_youtube_comments
from sentiment_analysis import analyze_sentiment
from save_data import save_excel

c:\Users\pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Scraping Comments

In [3]:
url = "https://www.youtube.com/watch?v=pgPUesOIgm0"
comments, timestamps, users = scrape_youtube_comments(url)

In [5]:
save_excel(comments, timestamps, users, filename="youtube_comments.xlsx")

Data saved!


In [6]:
if os.path.exists("youtube_comments.xlsx"):
    df = pd.read_excel("youtube_comments.xlsx")

In [7]:
df.head()

,Timestamp,User,Comment,Sentiment
0,1 year ago,@gcw2007,In other breaking news water is wet and fire i...,NaN
1,1 year ago,@Csharpflat5,Is anyone surprised by this really,NaN
2,1 year ago,@sammydsouza4379,seriously How dumb do you think we are I switc...,NaN
3,1 year ago,@carlosk8103,The actual news is not surprising to anyone Wh...,NaN
4,1 year ago,@TheJohnnyJohnny,Breaking news bic may have been living in the ...,NaN


# Sentiment Analysis

In [ ]:
for i in range(len(comments)):
    try:
        sentiment = analyze_sentiment(str(comments[i]))
        df.at[i, 'Sentiment'] = sentiment
        print(f"------- {i} -------")
    except Exception as e:
        print(f"Error processing comment")
        df.at[i, 'Sentiment'] = "error"

In [9]:
df.head()

,Timestamp,User,Comment,Sentiment
0,1 year ago,@gcw2007,In other breaking news water is wet and fire i...,negative
1,1 year ago,@Csharpflat5,Is anyone surprised by this really,negative
2,1 year ago,@sammydsouza4379,seriously How dumb do you think we are I switc...,negative
3,1 year ago,@carlosk8103,The actual news is not surprising to anyone Wh...,negative
4,1 year ago,@TheJohnnyJohnny,Breaking news bic may have been living in the ...,negative


In [12]:
# new_data_df = pd.read_excel("youtube_comments.xlsx")
# df = pd.concat([df, new_data_df], ignore_index=True)
df.to_excel("youtube_comments.xlsx", index=False)

# Logistic Regression

In [13]:
# Load CSV file
df = pd.read_excel("youtube_comments.xlsx")

X = df["Comment"].astype(str) 
y = df["Sentiment"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [15]:
reg = linear_model.LogisticRegression(max_iter=10000, random_state=0)
reg.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=10000, random_state=0)

In [16]:
y_pred = reg.predict(X_test_tfidf)

In [17]:
y_pred

array(['negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative'], dtype=object)

In [18]:
print(f"Logistic Regression model accuracy: {metrics.accuracy_score(y_test, y_pred) * 100:.2f}%")
print(f"Classification Report: ")
print(classification_report(y_test, y_pred))

Logistic Regression model accuracy: 81.82%
Classification Report: 
              precision    recall  f1-score   support

    negative       0.82      1.00      0.90        27
     neutral       0.00      0.00      0.00         2
    positive       0.00      0.00      0.00         4

    accuracy                           0.82        33
   macro avg       0.27      0.33      0.30        33
weighted avg       0.67      0.82      0.74        33



c:\Users\pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

# Saving Model

In [19]:
import joblib

joblib.dump(reg, "logistic_model.joblib")

['logistic_model.joblib']

# SQLite

In [20]:
import sqlite3

# Step 1: Load Excel file
df = pd.read_excel("youtube_comments.xlsx")

# Step 2: Connect to SQLite (creates a new DB file if it doesn't exist)
conn = sqlite3.connect("youtube_comments.db")

# Step 3: Write to a new table called 'comments'
df.to_sql("comments", conn, if_exists="replace", index=False)

# Step 4: Close the connection
conn.close()